# Knapsack Problem

__Inputs:__ Weight limit $W$, list of item weights $[w_1, \ldots, w_k]$, and list of item values $[v_1, \ldots, v_k]$.

__Output:__ Choose how many of each item to take $[n_1, \ldots, n_k]$ so that 
   1. Total weight is under the knapsack weight limit: $n_1 w_1 + \cdots + n_k w_k \leq W$.
   2. The value of stolen goods is maximized: $n_1 v_1 + \ldots + n_k v_k $ is max.

In [3]:
W = 200 #e
weights = [1, 5, 20, 35, 90] #energy
values = [15, 14.5, 19.2, 19.8, 195.2] #moves

## 1. Identify the optimal substructure

Suppose the current weight limit is $W$. Let us commit to stealing one of the available items and look at what is left to do.

   1. Suppose we commit to stealing item $j$.
   2. We now need to solve the same problem but for weight limit $W - w_j$. If the solution for this subproblem is obtained, then the original problem's solution is to take the solution for $W-w_j$ and append item $j$ to it.
   
We can thus see that the problem has optimal substructure.

## 2. Recurrence

$$\text{maxSteal}(W) = \max\ \left\{ \begin{array}{ll}
0 & \leftarrow\ \text{Choose to steal nothing and Quit!}\\
v_1 + \text{maxSteal}(W - w_1) & \leftarrow\ \text{Choose one unit of item}\ 1 \\
v_2 + \text{maxSteal}(W - w_2) & \leftarrow\ \text{Choose one unit of item}\ 2 \\
\vdots & \\
v_k + \text{maxSteal}(W - w_k) & \leftarrow\ \text{Choose one unit of item}\ k\\
\end{array} \right.$$

Base Case:

  * $\text{maxSteal}(0) = 0$ 
  * $\text{maxSteal}(W) = -\infty$ if $W < 0$.
 
 


In [4]:
def maxSteal(W, weights, values):
    if W == 0: 
        return 0
    if W < 0:
        return 100000
    k = len(weights)
    assert len(values) == k
    opts = [ values[i] + maxSteal(W - weights[i], weights, values) for i in range(k) ]
    return max(opts)

In [5]:
#WARNING: This will run for a very very long time.
print(maxSteal(25, weights, values))
print(maxSteal(W, weights, values))

100555.2


KeyboardInterrupt: 

## 3. Memoize

Memoization is very simple. We make a table $T[0], ... , T[W]$ for storing $\text{maxSteal}(j)$ for j ranging from $0$ to $W$.
The rest just follows the structure of the recurrence taking care to handle -ve values for weight separately.

## 4. Recover Solution

We store in a separate table $S[0], \ldots, S[W]$ which option provides us with the best value.


In [8]:
def maxSteal_memo(W, weights, values):
    # Initialize the tables
    T = [0]* (W+1)
    S = [-1]* (W+1)
    k = len(weights)
    assert len(values) == k
    for w in range(1, W+1):
        opts =  [  ( (values[i]+ T[ w - weights[i] ]), i )  for i in range(k) if w - weights[i] >= 0 ]
        opts.append( (1000000, -1) ) # In case opts was empty from the previous step.
        T[w], S[w] = min(opts)
    # This finishes the computation
#     stolen_item_ids = []
#     weight_remaining = W
#     while weight_remaining >= 0:
#         item_id = S[weight_remaining]
#         if item_id >= 0:
#             stolen_item_ids.append('Steal Item ID %d: weight = %d, value = %f' % (item_id, weights[item_id], values[item_id]) )
#             weight_remaining = weight_remaining - weights[item_id]
#         else:
#             break
#     return T[W], stolen_item_ids
    return T

In [9]:
print(maxSteal_memo(25, weights, values))
print(maxSteal_memo(W, weights, values))

[0, 15, 30, 45, 60, 14.5, 29.5, 44.5, 59.5, 74.5, 29.0, 44.0, 59.0, 74.0, 89.0, 43.5, 58.5, 73.5, 88.5, 103.5, 19.2, 34.2, 49.2, 64.2, 79.2, 33.7]
[0, 15, 30, 45, 60, 14.5, 29.5, 44.5, 59.5, 74.5, 29.0, 44.0, 59.0, 74.0, 89.0, 43.5, 58.5, 73.5, 88.5, 103.5, 19.2, 34.2, 49.2, 64.2, 79.2, 33.7, 48.7, 63.7, 78.7, 93.7, 48.2, 63.2, 78.2, 93.2, 108.2, 19.8, 34.8, 49.8, 64.8, 79.8, 34.3, 49.3, 64.3, 79.3, 94.3, 48.8, 63.8, 78.8, 93.8, 108.8, 63.3, 78.3, 93.3, 108.3, 123.3, 39.0, 54.0, 69.0, 84.0, 99.0, 53.5, 68.5, 83.5, 98.5, 113.5, 68.0, 83.0, 98.0, 113.0, 128.0, 39.6, 54.599999999999994, 69.6, 84.6, 99.6, 54.099999999999994, 69.1, 84.1, 99.1, 114.1, 68.6, 83.6, 98.6, 113.6, 128.6, 83.1, 98.1, 113.1, 128.1, 143.1, 58.8, 73.8, 88.8, 103.8, 118.8, 73.3, 88.3, 103.3, 118.3, 133.29999999999998, 87.8, 102.8, 117.8, 132.79999999999998, 147.79999999999998, 59.400000000000006, 74.39999999999999, 89.39999999999999, 104.39999999999999, 119.39999999999999, 73.89999999999999, 88.89999999999999, 103.899